In [1]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
import pandas as pd
import csv
import torch
import gc
from numba import cuda
from sklearn.metrics import accuracy_score

#### Define path to train data file and validation data file

In [2]:
train_path = r"data/train.jsonl"
validation_path = r"data/validation.jsonl"

#### Load data and encoding class labels

In [3]:
def load_data(file, relevant_cols):
    '''
    Load data from file, retrieve only the relevant columns into a Pandas DataFrame.
    Concatentate all text datas from relevent columns in to a "text" column in Data-
    Frame.
    
    Args:
        file: path to the data file
        relevant_cols: relevant columns (features) to be used
    
    Returns:
        df: a Pandas DataFrame containing the relevant data
    '''
    df = pd.read_json(file, lines=True)
    df["text"] = df["postText"].explode() + df["targetTitle"].explode()
    df = df.loc[:, relevant_cols]
    df.tags = df.tags.explode()
    df.tags = df.tags.apply(label_encoding, args=("", ))
    return df


def label_encoding(text, target=""):
    '''
    Encode class label from String to Integer
    
    Args:
        text: label that needs to be encoded
        target:
            default: None. 3 Labels will be encoded to 1,2, and 3 respectively.
            If one label is given, this label will be encoded as 1. The other 
            twos labels will be encoded as 0.
            
    Returns:
        Encoding according to label given.
    '''
    if target:
        if text == target:
            return 1
        else:
            return 0
    else:
        if text == "phrase":
            return 0
        elif text == "passage":
            return 1
        elif text == "multi":
            return 2
        else:
            return text    

#### Load Train Data
    We will use postText and targetTitle texts for our models.

In [4]:
relevant_columns = ["text", "tags"]
train_set = load_data(train_path, relevant_columns)
train_set

,text,tags
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",1
1,NASA sets date for full recovery of ozone hole...,0
2,This is what makes employees happy -- and it's...,0
3,Passion is overrated — 7 work habits you need ...,2
4,The perfect way to cook rice so that it's perf...,0
...,...,...
3195,Has Facebook's video explosion completely shak...,1
3196,Cop Is Eating At A Chili's When Teen Hands Him...,1
3197,5 popular myths about visible signs of aging t...,2
3198,You need to see this Twitter account that pred...,0


#### Load Validation Data

In [5]:
validation_set = load_data(validation_path, relevant_columns)
validation_set

,text,tags
0,Five Nights at Freddy’s Sequel Delayed for Wei...,1
1,Why Arizona Sheriff Joe Arpaio’s fate could ha...,2
2,Here’s how much you should be tipping your hai...,0
3,"""Harry Potter"" alums reunite for new movieAlan...",2
4,A man swallowed a microSD card and you won't b...,1
...,...,...
795,This is what happens when you leave a hotel cl...,1
796,This Texas GOP elector announces that he won't...,0
797,This beauty queen cured her acne with one diet...,2
798,WikiLeaks' Julian Assange Reported DeadWikiLea...,1


#### Set up configurations for training transformer model
    - Because of time constraints, two hyper parameters selected in validation is: learning rate and warm up ratio
    - Warm-up is a way to reduce the primacy effect of the early training examples. The learning rate is increased linearly over the warm-up period. This often leads to faster training time.

In [6]:
configurations = []

for learning_rate in [4e-6, 1e-5, 4e-5]:
    for warumup_ratio in [0.02, 0.06, 0.1]:
        configurations += [{
            "overwrite_output_dir": True,
            "num_train_epochs": 10,
            "learning_rate": learning_rate,
            "warmup_ratio": warumup_ratio,
            "best_model_dir": r"outputs/distilbert/best_model",
            "output_dir" : "outputs/"
        }]


#### Train model with the above given configurations.
    - The model being train is a distilled version of BERT, aply named distilBERT, which is smaller and faster #to train but at the cost of accuracy.
    - num_labels set to 3 for fine-tuning with our dataset.
    - ignore_mismatched_sizes=True is needed if the model_type and the model itself have some discrepancies.
    - delete model and flush CUDA memory after training each model to preven out of memory error.

In [7]:
results = []

for config in configurations:
    
    config["output_dir"] = r"outputs/distilbert/distilbert_" + str(config["learning_rate"]) + "_" + str(config["warmup_ratio"])
        
    model = ClassificationModel("bert", "distilbert-base-cased", num_labels=3, args=config, ignore_mismatched_sizes=True)
    model.train_model(train_set, eval_df=validation_set, acc=accuracy_score)
    train = model.eval_model(train_set, acc=accuracy_score)
    evaluation = model.eval_model(validation_set, acc=accuracy_score)
    
    results += [(config["learning_rate"], config["warmup_ratio"], train[0], evaluation[0])]
    model = None
    gc.collect()
    torch.cuda.empty_cache()

C:\Users\phanm\miniconda3\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\phanm\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield e

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.5.ffn.lin2.bias', 'distilbert.transformer.layer.5.output_layer_norm.weight', 'distilbert.transformer.layer.2.attention.k_lin.weight', 'distilbert.transformer.layer.3.sa_layer_norm.weight', 'distilbert.transformer.layer.1.ffn.lin2.weight', 'distilbert.transformer.layer.2.sa_layer_norm.weight', 'vocab_transform.bias', 'distilbert.transformer.layer.4.output_layer_norm.weight', 'distilbert.transformer.layer.3.attention.out_lin.bias', 'distilbert.transformer.layer.1.ffn.lin2.bias', 'distilbert.transformer.layer.1.output_layer_norm.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.5.ffn.lin1.bias', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.1.attention.out_lin.weight', 'distilbert.transformer.layer.3.output_layer_norm.weight', 'distilbert.transformer.lay

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
C:\Users\phanm\miniconda3\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/3200 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/3200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/400 [00:00<?, ?it/s]

C:\Users\phanm\miniconda3\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/800 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

#### Model Selection using Validation Set
    - The best model is selected using accuracy as validation metric.
    - All model results are then saved to a dataframe and a csv file.

In [9]:
results_dict = {"Model name" : [],
                "Learning rate" : [],
                "Warmup Ratio" : [],
                "acc" : [],
                "eval_loss" : []
               }
for config in configurations:
    model_name = str(config["learning_rate"]) + "_" + str(config["warmup_ratio"])
    with open(r"outputs/distilbert/distilbert_" + model_name + r"/eval_results.txt", "r+") as file:
        data = file.readlines()
        results_dict["Model name"].append("Distilbert " + model_name)
        results_dict["Learning rate"].append(config["learning_rate"])
        results_dict["Warmup Ratio"].append(config["warmup_ratio"])
        results_dict["acc"].append(data[0][6:-1])
        results_dict["eval_loss"].append(data[1][12:-1])
df = pd.DataFrame.from_dict(results_dict).sort_values("acc", ascending=False)
df.to_csv("distilbert_validation_results.csv")
df

,Model name,Learning rate,Warmup Ratio,acc,eval_loss
5,Distilbert 1e-05_0.1,0.000010,0.10,0.58,1.4178257846832276
4,Distilbert 1e-05_0.06,0.000010,0.06,0.5725,1.3830605363845825
8,Distilbert 4e-05_0.1,0.000040,0.10,0.565,1.625411958694458
1,Distilbert 4e-06_0.06,0.000004,0.06,0.55375,1.060385627746582
3,Distilbert 1e-05_0.02,0.000010,0.02,0.5525,1.3568671321868897
0,Distilbert 4e-06_0.02,0.000004,0.02,0.54375,1.0574987411499024
2,Distilbert 4e-06_0.1,0.000004,0.10,0.5375,1.1028425216674804
6,Distilbert 4e-05_0.02,0.000040,0.02,0.41875,1.040203857421875
7,Distilbert 4e-05_0.06,0.000040,0.06,0.41875,1.0390802001953126
